In [5]:
#импорт библиотек
import numpy as np
import random

In [6]:
#расстояние между точками
def dist(x1, y1, x2, y2):
    return np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.probability = []

#найдем центры кластеров
def calc_centers(k, m, points):
    result = []
    for i in range(0, k):
        n_x = 0
        n_y = 0
        d_x = 0
        d_y = 0

        for point in points:
            n_x += pow(point.probability[i], m) * point.x
            d_x += pow(point.probability[i], m)
            n_y += pow(point.probability[i], m) * point.y
            d_y += pow(point.probability[i], m)

        x_cluster_center = n_x / d_x
        y_cluster_center = n_y / d_y

        result.append(Point(x_cluster_center, y_cluster_center))

    return result


#считаем коэфф принадлежности к определенному кластеру
def membership_coeff(cluster_num, centers, point, m):
    result = 0
    distance= dist(centers[cluster_num].x, centers[cluster_num].y, point.x, point.y)
    for center in centers:
        result += pow(distance/ dist(center.x, center.y, point.x, point.y), 2 / (1 - m))
    return result


#нормализация коэффициента
def normalize_coeff(probability):
    sum = 0
    for p in probability:
        sum += p
    for i in range(0, len(probability)):
        probability[i] /= sum


#инициализация центроидов
def init_centroid(points, k, x_center, y_center):
    R = 0
    n = len(points)
    for i in range(0, n):
        r = dist(x_center, y_center, points[i].x, points[i].y)
        if r > R:
            R = r
    x_cc = [R * np.cos(2 * np.pi * i / k) + x_center for i in range(k)]
    y_cc = [R * np.sin(2 * np.pi * i / k) + y_center for i in range(k)]
    result = []
    for i in range(0, k):
        result.append(Point(x_cc[i], y_cc[i]))
    return result


#решающая функция
def decision_function(points, centers):
    function_result = 0
    for point in points:
        probability = point.probability
        for i in range(0, len(probability)):
            function_result += probability[i] * dist(point.x, point.y, centers[i].x, centers[i].y)
    return function_result


def init_cluster_membership(points, k):
    for point in points:
        for i in range(0, k):
            point.probability.append(0)


def c_means(points, m, k):
    first_iteration = True
    init_cluster_membership(points, k)

    x_center = np.mean(list(map(lambda e: e.x, points)))
    y_center = np.mean(list(map(lambda e: e.y, points)))

    current_decision = 1
    previous_decision = 0

    while abs(previous_decision - current_decision) > 0.2:
        previous_decision = current_decision
        if first_iteration:
            centers = init_centroid(points, k, x_center, y_center)
            first_iteration = False
        else:
            centers = calc_centers(k, m, points)
        for point in points:
            for i in range(0, len(centers)):
                point.probability[i] = membership_coeff(i, centers, point, m)
            normalize_coeff(point.probability)
            current_decision = decision_function(points, centers)

In [7]:
n = 20
k = 4
m = 1.5
points = [Point(random.randint(1, 100), random.randint(1, 100)) for i in range(n)]

c_means(points, m, k)

for point in points:
    print(str(point.x) + ":" + str(point.y), end=" ")
    print(point.probability)

21:77 [0.004762637560819676, 0.9867048016621773, 0.006122788557133589, 0.0024097722198694704]
46:17 [0.005615606860752421, 0.0043963999265576904, 0.0668142328424287, 0.9231737603702612]
68:1 [0.003993436482633964, 0.0013484234211778897, 0.006415759690478304, 0.9882423804057098]
50:56 [0.24005230851342052, 0.6394759155942017, 0.03794669975550171, 0.08252507613687612]
85:83 [0.9572793698181912, 0.03359849541503548, 0.0021774500299202083, 0.006944684736853296]
30:8 [0.0033867451155650844, 0.0044231257766514515, 0.9017330297607653, 0.0904570993470182]
82:52 [0.993603028401567, 0.001758822554604469, 0.00039720832523400436, 0.004240940718594655]
36:65 [0.006281349577460036, 0.9868046477110601, 0.004106359047105862, 0.002807643664373923]
37:82 [5.6946623269983354e-05, 0.9999146070540631, 1.5868440327519375e-05, 1.257788233924212e-05]
61:31 [0.026067987199359603, 0.006926250012087959, 0.010182967546612565, 0.9568227952419398]
85:96 [0.8154591804496828, 0.15421689108299483, 0.008849329308693208